In [2]:
from instruments import Voltmeter,Sourcemeter,VSourcemeter,MercuryiTC,MercuryiPS
from instrument_group import InstrumentGroup
import pyvisa
from time import sleep

### See available instruments

In [3]:
rm = pyvisa.ResourceManager()
inst = rm.list_resources()
print(inst)

### Initialize instruments and create the instrument group

In [ ]:
# Initialise all instruments and create the instrument group. Add instruments you need, comment out instruments not being used.

voltmeters = [
    ("A",Voltmeter('GPIB0::6::INSTR')),
    ("B",Voltmeter('GPIB0::12::INSTR')),
    ("C",Voltmeter('GPIB0::22::INSTR')),
    ("D",Voltmeter('GPIB0::25::INSTR')),
    ]
sourcemeters = [
    ("A",Sourcemeter('GPIB0::5::INSTR')),
    ("B",Sourcemeter('GPIB0::11::INSTR')),
    ]
Vsourcemeters = [
    ("A",VSourcemeter('GPIB0::23::INSTR')),
    ("B",VSourcemeter('GPIB0::24::INSTR')),
    ]
iTC = MercuryiTC('ASRL7::INSTR')
iPS = MercuryiPS('ASRL8::INSTR')
IG = InstrumentGroup(voltmeters, sourcemeters, Vsourcemeters, iTC, iPS)

### Run your script

In [ ]:
### Valid commands are:
# sleep(seconds)
# IG.ramp_T(filename,controller,temp,rate,threshold=0.05,timeout_hours=12) # controller is 'VTI', 'probe', or 'both', rate is in K/min